In [ ]:
from dataclasses import dataclass
import requests
import json
import pandas as pd


@dataclass
class Request:
    last_modified: int = 1698655128479
    page_offset: int = 0
    page_size: int = 1000

    def to_dict(self):
        return {
            "lastModified": self.last_modified,
            "page_offset": self.page_offset,
            "page_size": self.page_size
        }


def do_request(req: Request):
    url = "https://zen-cn-qa.kyligence.io/insight/api/v2/ai/export/backend_records"
    api_key = "5301de5bfb53b68a540be562ec0e4a1496dc85d18764c64a07e1e203a86b1d158a2e7a9be26850b212a8da23c175e665342de49d7a4a001"

    headers = {
        "Content-Type": "application/json",
        "Authorization": "Apikey " + api_key,
        "X-Language": "zh",
        "Accept-Language": "zh",
        "xdebug-user": "yuhui.liu",
        "assumed-tenant": "93029942410919939"
    }

    params = req.to_dict()

    print(f"Req Params: {params}")

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        api_res = json.loads(response.text)
        if len(api_res['data']) <= 0:
            print(api_res)
            return []
        items = api_res['data']['items']
        size = api_res['data']['size']
        page = api_res['data']['page']
        print(f"Fetch [{len(items)}] records, current page: [{page}] total size: [{size}]")
        return items
    else:
        print("请求失败，状态码:", response.status_code)

    return []

In [1]:
from typing import List
from evaluate import (
    Question,
    QuestionCategory,
    load_data,
    load_question_data,
    save_df_to_excel,
    transform_intention_data,
    transform_chart_analysis_data,
    transform_query_spec_data,
    transform_sql_data,
    transform_goal_report_data,
    transform_metric_analysis_data,
    transform_metric_recommend_data,
    questions_to_dataframe
)

In [ ]:
req = Request(page_size=5000, page_offset=4)
records = do_request(req)

ids = []
raw_inputs = []
raw_records = []

for i, record in enumerate(records):
    chat_req_msg = json.loads(record['chat_request_message'])

    ids.append(record['id'])
    raw_inputs.append(json.dumps(chat_req_msg, ensure_ascii=False, indent=2))
    raw_records.append(json.dumps(record, ensure_ascii=False, indent=2))

df = pd.DataFrame(columns=['id', 'raw_input', 'raw_record'])
df = pd.concat([df, pd.DataFrame({
    'id': ids,
    'raw_input': raw_inputs,
    'raw_record': raw_records,
})], ignore_index=False)

save_df_to_excel(df, f"data/chat_records_{req.page_offset}.xlsx")

In [3]:
import pandas as pd
import glob

# 获取所有Excel文件的文件名
excel_files = glob.glob("data/chat_records_*.xlsx")  # 替换为你的文件路径或者文件类型

# 创建一个空的DataFrame来存储合并后的数据
merged_data = pd.DataFrame()

# 循环读取每个Excel文件并将其合并到DataFrame中
for file in excel_files:
    df = pd.read_excel(file)
    merged_data = pd.concat([merged_data, df], ignore_index=True)

# 将合并后的数据保存到新的Excel文件中
merged_data.to_excel("data/chat_records_1113.xlsx", index=False)

print(f"[{len(merged_data)}] data were merged successfully.")

[25633] data were merged successfully.


In [4]:
def run_cleaning_pipeline(file_path: str) -> List[Question]:
    questions = load_data(file_path)
    questions = transform_intention_data(questions)
    questions = transform_chart_analysis_data(questions)
    questions = transform_query_spec_data(questions)
    questions = transform_sql_data(questions)
    questions = transform_goal_report_data(questions)
    questions = transform_metric_recommend_data(questions)
    questions = transform_metric_analysis_data(questions)
    save_df_to_excel(questions_to_dataframe(questions), f"data/question_v20.xlsx")
    return questions


question_list = run_cleaning_pipeline(f"data/chat_records_1113.xlsx")

failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: line 1 column 1 (char 0)
failed to convert to messages: Expecting value: lin

In [7]:
"""
转成 alpaca 微调格式
"""

import json
from byzerllm.utils.basic_llm_utils import MessageRole

all_questions = load_question_data(f"data/question_v20.xlsx", drop_duplicates=False)
alpaca_sft_data = []

for i, q in enumerate(all_questions):
    category = q.category
    if category != QuestionCategory.INTENTION.value:
        continue
    messages = q.messages
    if len(messages) <= 0:
        continue
    instruction = messages[0].content
    user_input = ""
    # user examples
    for msg in messages[1:]:
        role, content = msg.role, msg.content
        if role == MessageRole.USER:
            user_input = content
        elif role == MessageRole.ASSISTANT:
            alpaca_sft_data.append({
                "instruction": instruction,
                "input": user_input,
                "output": content
            })
    # user input
    alpaca_sft_data.append({
        "instruction": instruction,
        "input": user_input,
        "output": q.answer
    })

unique_alpaca_sft_data = [dict(t) for t in set([tuple(d.items()) for d in alpaca_sft_data])]

print(f"transform [{len(unique_alpaca_sft_data)}] alpaca intention data")

sft_data_json = json.dumps(unique_alpaca_sft_data, ensure_ascii=False, indent=2)
with open("data/alpha_sft_data_1113.json", "w") as file:
    file.write(sft_data_json)

load [23696] questions data from data/question_v20.xlsx
transform [3460] alpaca intention data
